Example from: [https://python.langchain.com/docs/integrations/chat/ollama_functions/](https://python.langchain.com/docs/integrations/chat/ollama_functions/)

In [1]:
import json
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.messages import HumanMessage
from typing import List

model = OllamaFunctions(model="llama3:70b", base_url='http://172.28.105.30/backend', format='json')

In [2]:
class Solution(BaseModel):
    temperature: float = Field(
        ..., description="The temperature of the solution creation")
    atmosphere: str = Field(
        ..., description="The atmosphere of the solution creation")
    method: str = Field(
        ..., description="The method of the solution creation")
    time: float = Field(
        ..., description="The time needed for the solution creation")
    solutes: List[str] = Field(
        ..., description="The solutes used in the solution")
    solute_masses: List[float] = Field(
        ..., description="The masses in miligramm of the solutes used in the solution")
    solvents: List[str] = Field(
        ..., description="The solvents used in the solution")
    solvent_volumes: List[float] = Field(
        ..., description="The volumes in mililiter of the solvents used in the solution")
    
class Scaling(BaseModel):
    powder: str = Field(
        ..., description="The name of the powder to be scaled")
    mass: float = Field(
        ..., description="The scaled mass of the powder")


In [3]:
test_msg = "Please create a solution of 2mg lead iodide as a solute and 100 ml ethanol as a solvent. The solution is produced at 25 degree celsius stirred for 15 minutes."
unclear_msg = "Please create a polyjuice potion."
scaling_msg = "Please create a scaling entry for 5mg of lead iodide."

In [4]:
# Prompt template
prompt = PromptTemplate.from_template(
    """
you should extract the information form the following text and create a structured output.
If information is missing you should fill it with null.
If something is unclear you should ask for clarification.

Human: {question}
AI:
"""
)


In [5]:
model = model.bind_tools(
    tools=[
        {
            "name": "solution_preparation",
            "description": "Schema for solution preparation",
            "parameters": Solution.schema(),
        },
        {
            "name": "powder_scaling",
            "description": "Schema for powder scaling",
            "parameters": Scaling.schema(),
        }
    ],
)

In [6]:
chain_with_tools = prompt | model

In [14]:
print(f'Asking: "{test_msg}"')
response_test = chain_with_tools.invoke(test_msg)
schema = response_test.additional_kwargs['function_call']['name']
instance = json.loads(response_test.additional_kwargs['function_call']['arguments'])
print(f'Schema used: {schema}')
print('Created instance:')
print(json.dumps(instance, indent=2))

Asking: "Please create a solution of 2mg lead iodide as a solute and 100 ml ethanol as a solvent. The solution is produced at 25 degree celsius stirred for 15 minutes."
Schema used: solution_preparation
Created instance:
{
  "temperature": 25,
  "atmosphere": null,
  "method": "stirred",
  "time": 15,
  "solutes": [
    "lead iodide"
  ],
  "solute_masses": [
    2
  ],
  "solvents": [
    "ethanol"
  ],
  "solvent_volumes": [
    100
  ]
}


In [16]:
print(f'Asking: "{scaling_msg}"')
response_scaling = chain_with_tools.invoke(scaling_msg)
schema = response_scaling.additional_kwargs['function_call']['name']
instance = json.loads(response_scaling.additional_kwargs['function_call']['arguments'])
print(f'Schema used: {schema}')
print('Created instance:')
print(json.dumps(instance, indent=2))

Asking: "Please create a scaling entry for 5mg of lead iodide."
Schema used: powder_scaling
Created instance:
{
  "powder": "lead iodide",
  "mass": 5
}
